<a href="https://colab.research.google.com/github/comsa33/AI_07_session_1_project_-/blob/main/ai_07_%E1%84%8B%E1%85%B5%E1%84%85%E1%85%AE%E1%84%8B%E1%85%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp, ttest_ind, chisquare, chi2_contingency
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

path = "/content/drive/MyDrive/vgames2.csv"

df_vgames = pd.read_csv(path, index_col=0)

df_vgames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


In [78]:
df_vgames.isna().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [79]:
cols = df_vgames.columns
for col in cols:
    missing_rate = df_vgames[col].isna().sum()/df_vgames.shape[0]
    if missing_rate > 0:
        print('[{}]의 결측치 비율은 {}% 입니다'.format(col, round(missing_rate*100,4)))

[Year]의 결측치 비율은 1.6327% 입니다
[Genre]의 결측치 비율은 0.3012% 입니다
[Publisher]의 결측치 비율은 0.3494% 입니다


In [80]:
df_new = df_vgames.fillna('Unknown')
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16598 non-null  object
 1   Platform     16598 non-null  object
 2   Year         16598 non-null  object
 3   Genre        16598 non-null  object
 4   Publisher    16598 non-null  object
 5   NA_Sales     16598 non-null  object
 6   EU_Sales     16598 non-null  object
 7   JP_Sales     16598 non-null  object
 8   Other_Sales  16598 non-null  object
dtypes: object(9)
memory usage: 1.3+ MB


In [81]:
df_new[cols[5]]

1        0.04
2        0.17
3           0
4        0.04
5        0.12
         ... 
16594    0.15
16595    0.01
16596    0.44
16597    0.05
16598       0
Name: NA_Sales, Length: 16598, dtype: object

In [82]:
def unify_num(string):
    if 'K' in string:
        new_string = string.replace('K', '')
        return float(new_string)*0.01
    elif 'M' in string:
        new_string = string.replace('M', '')
        return float(new_string)
    else:
        return float(string)

In [83]:
df_new[cols[5:]] = df_new[cols[5:]].applymap(unify_num)

In [84]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16598 non-null  object 
 3   Genre        16598 non-null  object 
 4   Publisher    16598 non-null  object 
 5   NA_Sales     16598 non-null  float64
 6   EU_Sales     16598 non-null  float64
 7   JP_Sales     16598 non-null  float64
 8   Other_Sales  16598 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.3+ MB


In [85]:
df_new.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0.00,0.00,0.00
2,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0.00,0.00,0.02,0.00
4,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0.00,0.00,0.00
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0.00,0.04
